<a href="https://colab.research.google.com/github/RuwaAbey/PyTorch-Learning/blob/main/Object%20Detection/Metrics/Intersection_over_Union.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import Libraries**

In [6]:
import torch

**Intersection over Union**

In [7]:
def intersection_over_union(boxes_preds, boxes, boxes_labels, box_format="midpoint"):

  '''
  Calucaltes intersection over union

  Parameters:
    boxes_preds(tensor) : Prediction of Bounding Boxes (BATCH SIZE, 4)
    boxes_lables(tesnor) : Correct labels of Bounding Boxes (BATCH SIZE, 4)
    box format (str): midpoint/corners, if boxes (x,y,w,h) or (x1,y1,x2,y2)

  Returns:
    tensor: Intersection over unioin for all examples
  '''

  #Boxes_pred shape in (N, 4) where N is the number of boxes
  #boxes_lables in (N, 4)]

  #Checking if the input boxes are in 'midpoint' format, and if so, convert them to corner coordinates (x1, y1, x2, y2).
  if box_format == "midpoint":
    # Predicted box conversion from (x_center, y_center, width, height) to (x1, y1, x2, y2)
    box1_x1 = boxes_preds[...,0:1] - boxes_preds[...,2:3]/2
    box1_y1 = boxes_preds[...,1:2] - boxes_preds[...,3:4]/2
    box1_x2 = boxes_preds[...,0:1] + boxes_preds[...,2:3]/2
    box1_y2 = boxes_preds[...,1:2] + boxes_preds[...,3:4]/2

    # Ground truth box conversion from (x_center, y_center, width, height) to (x1, y1, x2, y2)
    box2_x1 = boxes_labels[...,0:1] - boxes_labels[...,2:3]/2
    box2_y1 = boxes_labels[...,1:2] - boxes_labels[...,3:4]/2
    box2_x2 = boxes_labels[...,0:1] + boxes_labels[...,2:3]/2
    box2_y2 = boxes_labels[...,1:2] + boxes_labels[...,3:4]/2

  #If boxes are in 'corners' format, no conversion is needed.
  elif box_format == "corners":
    #box1 --> predicted box
    #Predicted boxes 4 cordinates
    # Predicted box (already in x1, y1, x2, y2)
    box1_x1 = boxes_preds[..., 0:1]  # x1 of predicted box
    box1_y1 = boxes_preds[..., 1:2]  # y1 of predicted box
    box1_x2 = boxes_preds[..., 2:3]  # x2 of predicted box
    box1_y2 = boxes_preds[..., 3:4]  # y2 of predicted box

    #box2 --> Correct box
    #Correct boxes 4 cordinates
    #Ground truth box (already in x1, y1, x2, y2)
    box2_x1 = boxes_labels[..., 0:1]  # x1 of ground truth box
    box2_y1 = boxes_labels[..., 1:2]  # y1 of ground truth box
    box2_x2 = boxes_labels[..., 2:3]  # x2 of ground truth box
    box2_y2 = boxes_labels[..., 3:4]  # y2 of ground truth box

  #Taking the cordinates of the intersection box
  # Calculating the coordinates of the intersection box
  # (x1_intersection, y1_intersection) is the top-left corner of the intersected area
  # (x2_intersection, y2_intersection) is the bottom-right corner of the intersected area
  x1 = torch.max(box1_x1, box2_x1)  # Take the maximum of the leftmost x-coordinates
  y1 = torch.max(box1_y1, box2_y1)  # Take the maximum of the topmost y-coordinates
  x2 = torch.min(box1_x2, box2_x2)  # Take the minimum of the rightmost x-coordinates
  y2 = torch.min(box1_y2, box2_y2)  # Take the minimum of the bottommost y-coordinates

  #Area of intersection box
  intersection_area = (x2 - x1).clamp(0) *(y2 - y1).clamp(0)
  '''
  .clamp(0) prevents negative values in case the boxes don't intersect.
  If the two bounding boxes do not overlap, the (x2 - x1) or (y2 - y1) values would be negative,
  so we clamp the result to 0 to represent no intersection.
  '''

  # Calculate the areas of both bounding boxes for the unoin calcualtion
  box1_area = abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
  box2_area = abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))

  #Calculating the union area (# Calculate the union area (area of both boxes minus the area of their intersection))
  union_area = box1_area + box2_area - intersection + 1e-6   #1e-6 for numerical stability
  '''
  The small constant (1e-6) is added to the denominator to avoid division by zero,
  especially in cases where the union area is extremely small (e.g., if both boxes are identical).
  This ensures that the IoU computation remains stable and avoids producing extremely large or undefined values.
  '''

  # Return the IoU (ratio of the intersection area to the union area)
  return intersection_area / union_area